### **Reconhecedor de padrões baseado em LZW**

#### Pré-processamento das imagens

##### 1. Retirada do cabeçalho dos dados

In [9]:
import os

In [2]:
# Diretório de origem
diretorio_origem = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces"

# Diretório de destino
diretorio_destino = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_sc"

In [3]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino
    caminho_destino_subpasta = os.path.join(diretorio_destino, subpasta)
    os.makedirs(caminho_destino_subpasta, exist_ok=True)

    # Percorre os arquivos de imagem na subpasta atual
    for arquivo in os.listdir(caminho_subpasta):
        if arquivo.endswith(".pgm"):
            caminho_arquivo_origem = os.path.join(caminho_subpasta, arquivo)

            # Abre o arquivo de origem em modo binário
            with open(caminho_arquivo_origem, "rb") as arquivo_origem:
                dados = arquivo_origem.read()

            # Remove as três primeiras linhas
            novos_dados = dados.split(b"\n", 3)[-1]

            # Cria o caminho do arquivo de destino
            caminho_arquivo_destino = os.path.join(caminho_destino_subpasta, arquivo)

            # Salva os novos dados no arquivo de destino em modo binário
            with open(caminho_arquivo_destino, "wb") as arquivo_destino:
                arquivo_destino.write(novos_dados)

#### Geração dos modelos

##### 1. Divisão aleatória do banco entre instâncias de "treinamento" e "teste"

In [4]:
import shutil
import random

In [2]:
# Diretório de origem
diretorio_origem = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_sc"

# Diretório de destino para treinamento
diretorio_destino_treinamento = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_div\train"

# Diretório de destino para teste
diretorio_destino_teste = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_div\test"

# Porcentagem de imagens para treinamento (90%)
porcentagem_treinamento = 0.9

In [6]:
# Exclui os diretórios de destino, se já existirem
if os.path.exists(diretorio_destino_treinamento):
    shutil.rmtree(diretorio_destino_treinamento)
if os.path.exists(diretorio_destino_teste):
    shutil.rmtree(diretorio_destino_teste)

In [7]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta_origem = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino para treinamento
    caminho_subpasta_destino_treinamento = os.path.join(diretorio_destino_treinamento, subpasta)
    os.makedirs(caminho_subpasta_destino_treinamento)

    # Cria a subpasta correspondente no diretório de destino para teste
    caminho_subpasta_destino_teste = os.path.join(diretorio_destino_teste, subpasta)
    os.makedirs(caminho_subpasta_destino_teste)

    # Lista os arquivos na subpasta atual
    arquivos = os.listdir(caminho_subpasta_origem)
    random.shuffle(arquivos)  # Embaralha a ordem dos arquivos

    # Calcula o número de arquivos para treinamento e teste
    num_arquivos_treinamento = int(len(arquivos) * porcentagem_treinamento)
    num_arquivos_teste = len(arquivos) - num_arquivos_treinamento

    # Divide os arquivos entre treinamento e teste
    arquivos_treinamento = arquivos[:num_arquivos_treinamento]
    arquivos_teste = arquivos[num_arquivos_treinamento:]

    # Move os arquivos para as respectivas pastas de treinamento e teste
    for arquivo in arquivos_treinamento:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_treinamento, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

    for arquivo in arquivos_teste:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_teste, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

##### 2. Construção dos modelos/dicionários para cada classe

In [10]:
dict_size = 256

In [11]:
def lzw_encode(input_file, dictionary, max_dict_size):
    current_sequence = bytes()
    global dict_size

    with open(input_file, "rb") as file:
        while True:
            symbol = file.read(1)
            if not symbol:
                break

            next_sequence = current_sequence + symbol
            if next_sequence in dictionary:
                current_sequence = next_sequence
            else:
                if dict_size < 2 ** max_dict_size:
                    dictionary[next_sequence] = dict_size
                    dict_size += 1
                current_sequence = symbol

In [12]:
def encode_images_in_folders(folder_path, max_dict_size):
    dictionaries = []

    # Verifica se o tamanho máximo do dicionário está dentro do intervalo válido
    if not (9 <= max_dict_size <= 16):
        raise ValueError("O tamanho máximo do dicionário deve estar entre 9 e 16.")

    # Percorre todas as pastas
    for folder_name in os.listdir(folder_path):
        folder_dir = os.path.join(folder_path, folder_name)
        dictionary = {bytes([i]): i for i in range(256)}

        # Codifica as imagens na pasta atual
        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)

            # Realiza a codificação LZW para cada imagem
            lzw_encode(image_path, dictionary, max_dict_size)
        
        global dict_size
        dict_size = 256

        # Adiciona o dicionário da pasta à lista de dicionários
        dictionaries.append((folder_name, dictionary))

    return dictionaries

In [13]:
# Diretório das pastas de imagens
folder_path = diretorio_destino_treinamento
# Tamanho máximo do dicionário (2^k)
max_dict_size = 16
# Codifica as imagens em todas as pastas e obtém a lista de dicionários
all_dictionaries = encode_images_in_folders(folder_path, max_dict_size)

In [14]:
folder_name, dictionary = all_dictionaries[7]
print(f"Dicionário da pasta '{folder_name}'")
print(f"Tamanho: {len(dictionary)}")

Dicionário da pasta 's16'
Tamanho: 49937


##### 3. Classificação das imagens de teste

In [15]:
def lzw_encode(input_file, dictionary):
    current_sequence = bytes()
    encoded_data = []

    with open(input_file, "rb") as file:
        for symbol in file.read():
            next_sequence = current_sequence + bytearray([symbol])
            if bytes(next_sequence) in dictionary[1]:
                current_sequence = next_sequence
            else:
                encoded_data.append(dictionary[1][bytes(current_sequence)])
                current_sequence = bytearray([symbol])

    encoded_data.append(dictionary[1][bytes(current_sequence)])
    return encoded_data

In [16]:
import struct

In [17]:
def encode_images_in_folders(folder_path, dictionaries, output_folder):

    os.makedirs(output_folder, exist_ok=True)

    for folder_name in os.listdir(folder_path):
        folder_dir = os.path.join(folder_path, folder_name)
        encoded_data = []

        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)

            for dictionary in dictionaries:
                encoded_data = lzw_encode(image_path, dictionary)
                encoded_file_path = os.path.join(output_folder, f"{folder_name}_{image_name}_dict_{dictionary[0]}.bin")
                
                with open(encoded_file_path, "wb") as output_file:
                    for value in encoded_data:
                        output_file.write(struct.pack("H", value))

In [18]:
# Definir os caminhos das pastas de entrada, onde estão localizadas as imagens
folder_path = diretorio_destino_teste

# Caminho para salvar os arquivos binários gerados para cada valor de k
output_folder = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\output\k_16"

# Lista de tuplas contendo os dicionários e suas classes associadas
dictionaries = all_dictionaries

In [104]:
# Codificar as imagens nas pastas usando os dicionários especificados
encode_images_in_folders(folder_path, dictionaries, output_folder)

##### 4. Avaliação do modelo para cada valor de k

In [19]:
def find_smallest_files(folder_path, batch_size):
    smallest_files = {}

    # Obtém a lista de todos os arquivos da pasta e ordena em ordem alfabética
    all_files = sorted(os.listdir(folder_path))

    # Itera sobre os lotes de imagens
    for batch_start in range(0, len(all_files), batch_size):
        batch_end = batch_start + batch_size
        batch_files = []

        # Obtém os nomes dos arquivos do lote atual
        for file_name in all_files[batch_start:batch_end]:
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                batch_files.append(file_path)

        if batch_files:
            smallest_file = min(batch_files, key=lambda x: os.path.getsize(x))
            batch_number = batch_start // batch_size
            smallest_files[batch_number] = os.path.basename(smallest_file)

    # Verifica se os menores arquivos possuem o prefixo correspondente ao sufixo "si.bin"
    matching_files = {}
    for batch_number, file_name in smallest_files.items():
        prefix = file_name.split("_")[0]
        suffix = f"{prefix}.bin"
        if file_name.endswith(suffix):
            matching_files[batch_number] = file_name

    return matching_files

In [20]:
# Exemplo de uso
folder_path = output_folder
batch_size = 40
matching_files = find_smallest_files(folder_path, batch_size)

for batch_number, file_name in matching_files.items():
    print(f"No lote {batch_number + 1}: menor arquivo é {file_name} e indica acerto")

No lote 1: menor arquivo é s10_8.pgm_dict_s10.bin e indica acerto
No lote 2: menor arquivo é s11_3.pgm_dict_s11.bin e indica acerto
No lote 3: menor arquivo é s12_5.pgm_dict_s12.bin e indica acerto
No lote 4: menor arquivo é s13_2.pgm_dict_s13.bin e indica acerto
No lote 5: menor arquivo é s14_8.pgm_dict_s14.bin e indica acerto
No lote 6: menor arquivo é s15_10.pgm_dict_s15.bin e indica acerto
No lote 7: menor arquivo é s16_3.pgm_dict_s16.bin e indica acerto
No lote 8: menor arquivo é s17_8.pgm_dict_s17.bin e indica acerto
No lote 9: menor arquivo é s18_10.pgm_dict_s18.bin e indica acerto
No lote 10: menor arquivo é s19_6.pgm_dict_s19.bin e indica acerto
No lote 11: menor arquivo é s1_5.pgm_dict_s1.bin e indica acerto
No lote 12: menor arquivo é s20_1.pgm_dict_s20.bin e indica acerto
No lote 13: menor arquivo é s21_1.pgm_dict_s21.bin e indica acerto
No lote 14: menor arquivo é s22_9.pgm_dict_s22.bin e indica acerto
No lote 15: menor arquivo é s23_10.pgm_dict_s23.bin e indica acerto
No 

In [21]:
result = round(len(matching_files.items()) / 40, 4)

print(f"Acurácia para '{folder_path}': {result * 100}%")

Acurácia para 'C:\Users\franklin.coelho\Documents\encode-decode-lzw\output\k_16': 90.0%
